# Rf 2 0 4_Extrangefit
 'ADDITION AND CONVOLUTION' RooFit tutorial macro #204

 Extended maximum likelihood fit with alternate range definition
 for observed number of events.
 If multiple ranges are used, or only a part of the data is fitted,
 it is advisable to use a RooAddPdf to extend the model. See tutorial
 204a.



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:43 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooChebychev.h"
#include "RooAddPdf.h"
#include "RooExtendPdf.h"
#include "RooFitResult.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Setup component pdfs
 ---------------------------------------

Declare observable x

In [3]:
RooRealVar x("x", "x", 0, 10);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create two gaussian pdfs g1(x,mean1,sigma) anf g2(x,mean2,sigma) and their parameters

In [4]:
RooRealVar mean("mean", "mean of gaussians", 5);
RooRealVar sigma1("sigma1", "width of gaussians", 0.5);
RooRealVar sigma2("sigma2", "width of gaussians", 1);

RooGaussian sig1("sig1", "Signal component 1", x, mean, sigma1);
RooGaussian sig2("sig2", "Signal component 2", x, mean, sigma2);

Build chebychev polynomial p.d.f.

In [5]:
RooRealVar a0("a0", "a0", 0.5, 0., 1.);
RooRealVar a1("a1", "a1", 0.2, 0., 1.);
RooChebychev bkg("bkg", "Background", x, RooArgSet(a0, a1));

Sum the signal components into a composite signal p.d.f.

In [6]:
RooRealVar sig1frac("sig1frac", "fraction of component 1 in signal", 0.8, 0., 1.);
RooAddPdf sig("sig", "Signal", RooArgList(sig1, sig2), sig1frac);

Construct extended comps with range spec
 ------------------------------------------------------------------------------

Define signal range in which events counts are to be defined

In [7]:
x.setRange("signalRange", 4, 6);

[#1] INFO:Eval -- RooRealVar::setRange(x) new range named 'signalRange' created with bounds [4,6]


Associated nsig/nbkg as expected number of events with sig/bkg _in_the_range_ "signalrange"

In [8]:
RooRealVar nsig("nsig", "number of signal events in signalRange", 500, 0., 10000) ;
RooRealVar nbkg("nbkg", "number of background events in signalRange", 500, 0, 10000) ;

Use addpdf to extend the model:

In [9]:
RooAddPdf  model("model","(g1+g2)+a", RooArgList(bkg,sig), RooArgList(nbkg,nsig)) ;

Clone these models here because the interpretation of normalisation coefficients changes
 when different ranges are used:

In [10]:
RooAddPdf model2(model);
RooAddPdf model3(model);

Sample data, fit model
 -------------------------------------------

Generate 1000 events from model so that nsig,nbkg come out to numbers <<500 in fit

In [11]:
RooDataSet *data = model.generate(x, 1000);


auto canv = new TCanvas("Canvas", "Canvas", 1500, 600);
canv->Divide(3,1);

Fit full range
 -------------------------------------------

In [12]:
canv->cd(1);

Perform unbinned ml fit to data, full range

In [13]:
RooFitResult* r = model.fitTo(*data,Save()) ;
r->Print() ;

[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (sig1,sig2)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (bkg)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a0           5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 a1           2.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     3 nbkg         5.00000e+02  2.50000e+02    0.00000e+00  1.00000e+04
     4 nsig         5.00000e+02  2.50000e+02    0.00000e+00  1.00000e+04
     5 sig1frac     8.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
 **********
 **    3 **SET ER